In [1]:
import sys
sys.path.insert(1, 'data_utils')
sys.path.insert(1, 'models/')

import open3d as o3d
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from modelnet_reg_utils import ModelNet40Data, RegistrationData
from torch.utils.data import DataLoader
from tqdm import tqdm
import copy
from scipy.spatial.transform import Rotation
import transforms3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
BATCH_SIZE = 16
testset = RegistrationData(ModelNet40Data(train=False, download=True), is_testing=True, 
                           angle_range=90, translation_range=1, add_noise=False, shuffle_points=False)
test_loader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True, num_workers=2)

In [3]:
def display_open3d(template, source, transformation):
	"""
	Display the template, source and transformed source point clouds using open3d
	The Red and Blue point clouds should overlap perfectly if the transformation is correct
	The Green point cloud should be far away from the Red and Blue point clouds"""
	template_ = o3d.geometry.PointCloud()
	source_ = o3d.geometry.PointCloud()
	template_.points = o3d.utility.Vector3dVector(template)
	source_.points = o3d.utility.Vector3dVector(source)
	transformed_source_ = copy.deepcopy(source_)
	transformed_source_ = transformed_source_.transform(transformation)
	template_.paint_uniform_color([1, 0, 0]) # Red
	source_.paint_uniform_color([0, 1, 0]) # Green
	transformed_source_.paint_uniform_color([0, 0, 1]) #Blue
	o3d.visualization.draw_geometries([template_, source_, transformed_source_])

In [4]:
def convert_to_trans(R,t):
    """
    convert rotation matrix and translation vector to transformation matrix
    """
    T = np.zeros((4,4))
    T[:3,:3] = R
    T[:3,3] = t
    T[3,3] = 1
    return T

In [5]:
def inverse(R):
    """
    inverse of rotation matrix
    """
    return R.T

In [6]:
def evaluate_icp(max_iteration, threshold, test_loader):
    r_mse = 0
    t_mse = 0
    r_mae = 0
    t_mae = 0
    idx = []
    r_idx = []
    count = 0
    for j, data in enumerate(tqdm(test_loader)):
        template, source, igt, igt_R, igt_t = data
        template = template.detach().numpy()
        source = source.detach().numpy()
        igt = igt.detach().numpy()
        igt_R = igt_R.detach().numpy()
        igt_t = igt_t.detach().numpy()
        for i in range(template.shape[0]):
            gt_t  = igt_t[i] - np.mean(igt_t[i], axis=0)
            gt_R = igt_R[i]
            src = template[i] - np.mean(template[i], axis=0)
            tgt = source[i] - np.mean(source[i], axis=0)
            gt_tsf = convert_to_trans(gt_R,gt_t)
            pred_tsf = o3d.pipelines.registration.registration_icp(
                        o3d.geometry.PointCloud(o3d.utility.Vector3dVector(src)), o3d.geometry.PointCloud(o3d.utility.Vector3dVector(tgt)), threshold, np.eye(4),
                        o3d.pipelines.registration.TransformationEstimationPointToPoint(),
                        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration = max_iteration)).transformation
            R_pred = pred_tsf[:3,:3].copy()
            pred_t = pred_tsf[:3,3].copy()
            #Displays the point clouds and the transformation
            # display_open3d(tgt, src, gt_tsf)
            # display_open3d(tgt, src, pred_tsf)

            #Computes the metrics
            
            #convert rotations to euler angles
            gt_euler = Rotation.from_matrix(gt_R).as_euler('zyx', degrees=True)
            pred_euler = Rotation.from_matrix(R_pred).as_euler('zyx', degrees=True)

            #compute the translation error
            t_mse += np.mean(np.square(gt_t - pred_t))
            t_mae += np.mean(np.abs(gt_t - pred_t))
            # t_rmse += np.sqrt(t_mse)

            #compute the rotation error
            # c_r_mae = np.mean(np.abs(gt_euler - pred_euler))
            r_mse += np.mean(np.square(gt_euler - pred_euler))
            r_mae += np.mean(np.abs(gt_euler - pred_euler))
            # r_rmse += np.sqrt(r_mse)
            
            #compute irotropic error
            # error_mat = np.dot(gt_R.T , R_pred)
            # _, angle = transforms3d.axangles.mat2axangle(error_mat)
            # r_error1 = abs(angle*(180/np.pi))
            # r_error = np.arccos(np.clip((np.trace(error_mat)-1)/2, -1, 1))
            # r_error = r_error*180/np.pi
            # if c_t_mae > 5:
            #     idx.append((j,i))
            # if r_error > 5:
            #     r_idx.append((j,i))
            count += 1
    metrics = {
        'r_mse': r_mse/count,
        't_mse': t_mse/count,
        'r_mae': r_mae/count,
        't_mae': t_mae/count,
        # 'r_rmse': r_rmse/count,
        # 't_rmse': t_rmse/count
    }
    return metrics, idx, r_idx


In [7]:
eval, idx, r_idx = evaluate_icp(30,1, test_loader)

100%|██████████| 154/154 [01:35<00:00,  1.61it/s]


In [8]:
eval

{'r_mse': 2301.4800888305326,
 't_mse': 0.0020776623815255244,
 'r_mae': 27.834655520088077,
 't_mae': 0.021719972785661224}

In [68]:
threshold = 1
max_iteration = 30
for j, data in enumerate(tqdm(test_loader)):
    template, source, igt, igt_R, igt_t = data
    template = template.detach().numpy()
    source = source.detach().numpy()
    igt = igt.detach().numpy()
    igt_R = igt_R.detach().numpy()
    igt_t = igt_t.detach().numpy()
    for i in range(template.shape[0]):
        if (j,i) in idx and (j,i) in r_idx:
            gt_t  = igt_t[i] - np.mean(igt_t[i], axis=0)
            gt_R = igt_R[i]
            src = template[i] - np.mean(template[i], axis=0)
            tgt = source[i] - np.mean(source[i], axis=0)
            gt_tsf = convert_to_trans(gt_R,gt_t)
            pred_tsf = o3d.pipelines.registration.registration_icp(
                        o3d.geometry.PointCloud(o3d.utility.Vector3dVector(src)), o3d.geometry.PointCloud(o3d.utility.Vector3dVector(tgt)), threshold, np.eye(4),
                        o3d.pipelines.registration.TransformationEstimationPointToPoint(),
                        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration = max_iteration)).transformation
            R_pred = pred_tsf[:3,:3].copy()
            pred_t = pred_tsf[:3,3].copy()
            #Displays the point clouds and the transformation
            # display_open3d(tgt, src, gt_tsf)
            display_open3d(tgt, src, pred_tsf)
        else:
            continue

  1%|          | 1/154 [03:47<9:40:40, 227.72s/it]


KeyboardInterrupt: 